Trabalhando com os dois arquivos Netfix e Amazon

**AMAZON**

In [ ]:
!pip install pyspark

     |████████████████████████████████| 281.3 MB 37 kB/s 
     |████████████████████████████████| 198 kB 42.6 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=2762967e6ef3f44aee55572529c51a10bcccccf873ebe14eb8a592ca07e86081
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.window import Window
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd


In [ ]:
spark = (
    SparkSession.builder
      .master('local')
      .appName('dataframe_withcolumn')
      .config('spark.ui.port', '4050')
      .getOrCreate()
)

Definindo esquema de cabeçalho e configurações da tabela

In [ ]:
esquema = ["id", "tipo", "titulo", "país", "data_adicionado","ano_lancamento","classificacao","duracao","genero"]
amazon_df = (spark
       .read
       .format("csv")
       .option("header", "true")
       .option("inferschema", "True")
       .option("delimiter", ",")
       .load(r"/content/ProjetoFinal_12/Amazon2021_tratado1_(8).csv").toDF(*esquema)
       )

In [ ]:
spark

In [ ]:
amazon_df.printSchema()

root
 |-- id: string (nullable = true)
 |-- tipo: string (nullable = true)
 |-- titulo: string (nullable = true)
 |-- país: string (nullable = true)
 |-- data_adicionado: string (nullable = true)
 |-- ano_lancamento: integer (nullable = true)
 |-- classificacao: string (nullable = true)
 |-- duracao: string (nullable = true)
 |-- genero: string (nullable = true)



In [ ]:
amazon_df.show(10)

+---+-----+--------------------+--------------+---------------+--------------+-------------+-------+--------------------+
| id| tipo|              titulo|          país|data_adicionado|ano_lancamento|classificacao|duracao|              genero|
+---+-----+--------------------+--------------+---------------+--------------+-------------+-------+--------------------+
| s1|Filme|    A Grande Sedução|        Canada|     2021-03-30|          2014| Nao avaliado|113 min|      Comedia, Drama|
| s2|Filme|  Cuide da boa noite|         India|     2021-03-30|          2018|          13+|110 min|Drama, Internacional|
| s3|Filme|Segredos da Decepção|Estados Unidos|     2021-03-30|          2017| Nao avaliado| 74 min|Acao, Drama, Susp...|
| s4|Filme|Rosa: Mantendo-se...|Estados Unidos|     2021-03-30|          2014| Nao avaliado| 69 min|        Documentario|
| s5|Filme| Criador de Monstros|   Reino Unido|     2021-03-30|          1989| Nao avaliado| 45 min|     Drama, Fantasia|
| s6|Filme|Living With D


 A Amazon Prime disponibiliza aproximadamente 10000 itens, sendo que destes podemos observar

 Quantos filmes e quantas Séries a Amazon disponibiliza

In [ ]:
amazon_df.groupby(F.col("tipo")).count().show()

+-----+-----+
| tipo|count|
+-----+-----+
|Serie| 1854|
|Filme| 7814|
+-----+-----+



Separamos os dados que dizem respeito ao público Infantil

In [ ]:
dfinfantil = amazon_df.filter(F.col('classificacao')=='Infantil')

Separamos os dados que correspondem ao público adulto

In [ ]:
dfadultos = amazon_df.filter(F.col('classificacao')=='18+')

Mostra quantos lançamentos teve em cada ano   Demonstrando um aumento grande na produção ou disponibilização de novos filmes.

In [ ]:
maislanc=amazon_df.groupby(F.col('ano_lancamento')).count().orderBy(F.col('ano_lancamento')).show()

+--------------+-----+
|ano_lancamento|count|
+--------------+-----+
|          1920|    3|
|          1922|    2|
|          1923|    1|
|          1924|    1|
|          1925|    8|
|          1926|    2|
|          1927|    1|
|          1929|    4|
|          1930|    5|
|          1931|    3|
|          1932|    8|
|          1933|    5|
|          1934|   12|
|          1935|   15|
|          1936|   29|
|          1937|   24|
|          1938|   15|
|          1939|   21|
|          1940|   16|
|          1941|   22|
+--------------+-----+
only showing top 20 rows



In [ ]:
w1 = Window.partitionBy(F.col('ano_lancamento')).orderBy('ano_lancamento')
amazon_df.groupby(F.col('ano_lancamento')).count()
amazon_df.withColumn('rank', F.rank().over(w1))

DataFrame[id: string, tipo: string, titulo: string, país: string, data_adicionado: string, ano_lancamento: int, classificacao: string, duracao: string, genero: string, rank: int]

Aumento significativo nos ultimos anos

In [ ]:
amazon_df.filter(F.col('ano_lancamento').like('2018')).count(),

(623,)

In [ ]:
amazon_df.filter(F.col('ano_lancamento').like('2019')).count(),

(929,)

In [ ]:
Am_lanc_20=amazon_df.filter(F.col('ano_lancamento').like('2020')).count()

In [ ]:
Am_lanc_21=amazon_df.groupby('ano_lancamento')
amazon_df.filter(F.col('ano_lancamento').like('2021')).count()

1442

Filtrando por tipo de programação

In [ ]:
amazon_df.select(F.col("titulo"),F.col("tipo"),F.col("genero")).filter("tipo = 'Filme'").show(10)


+--------------------+-----+--------------------+
|              titulo| tipo|              genero|
+--------------------+-----+--------------------+
|    A Grande Sedução|Filme|      Comedia, Drama|
|  Cuide da boa noite|Filme|Drama, Internacional|
|Segredos da Decepção|Filme|Acao, Drama, Susp...|
|Rosa: Mantendo-se...|Filme|        Documentario|
| Criador de Monstros|Filme|     Drama, Fantasia|
|Living With Dinos...|Filme|  Fantasia, Infantil|
|           Hired Gun|Filme|Documentario, Con...|
|        Grease Live!|Filme|             Comedia|
|     Global Meltdown|Filme|Acao, Ficcao Cien...|
|      David's Mother|Filme|               Drama|
+--------------------+-----+--------------------+
only showing top 10 rows



In [ ]:
amazon_df.select(F.col("ano_lancamento"),F.col("titulo"),F.col("tipo"),F.col("genero")).filter("tipo = 'Serie'").show()

+--------------+--------------------+-----+--------------------+
|ano_lancamento|              titulo| tipo|              genero|
+--------------+--------------------+-----+--------------------+
|          2008|          Zoo Babies|Serie|Infantil, Conteud...|
|          2020|Zoë Coombs Marr: ...|Serie|Comedia, Entrevis...|
|          2001|          Zoboomafoo|Serie|            Infantil|
|          2020|Zo Zo Zombie: Min...|Serie|Acao, Anime, Comedia|
|          2020|        ZeroZeroZero|Serie|              Series|
|          2020|  Zahara: The Return|Serie|              Series|
|          2017|Z: The Beginning ...|Serie|      Comedia, Drama|
|          2015|     Yu-Gi-Oh! Zexal|Serie|Animacao, Anime, ...|
|          2018|     Yu-Gi-Oh! ARC-V|Serie|Acao, Aventura, A...|
|          2011|      Yu-Gi-Oh! 5D's|Serie|     Anime, Infantil|
|          2013|Your Bleeped Up B...|Serie|              Series|
|          2002|         Young Lions|Serie|     Drama, Suspense|
|          2018|Yoga Ther

Criamos uma coluna (Aplicativo) para identificar os seviços de específicos desta tabela preparando para a junção em uma tabela unificada.

In [ ]:
from pyspark.sql.functions import regexp_replace
amazon_df = amazon_df.withColumn('tempo', regexp_replace('duracao', 'min', ''))

In [ ]:
amazon_df.show()

+---+-----+--------------------+--------------+---------------+--------------+-------------+-----------+--------------------+-----------+
| id| tipo|              titulo|          país|data_adicionado|ano_lancamento|classificacao|    duracao|              genero|      tempo|
+---+-----+--------------------+--------------+---------------+--------------+-------------+-----------+--------------------+-----------+
| s1|Filme|    A Grande Sedução|        Canada|     2021-03-30|          2014| Nao avaliado|    113 min|      Comedia, Drama|       113 |
| s2|Filme|  Cuide da boa noite|         India|     2021-03-30|          2018|          13+|    110 min|Drama, Internacional|       110 |
| s3|Filme|Segredos da Decepção|Estados Unidos|     2021-03-30|          2017| Nao avaliado|     74 min|Acao, Drama, Susp...|        74 |
| s4|Filme|Rosa: Mantendo-se...|Estados Unidos|     2021-03-30|          2014| Nao avaliado|     69 min|        Documentario|        69 |
| s5|Filme| Criador de Monstros|  

In [ ]:
amazon_df.groupby(F.col('duracao')).count().show()


+-------------+-----+
|      duracao|count|
+-------------+-----+
|      100 min|  126|
|      153 min|   17|
|       56 min|   42|
|       71 min|   52|
|       13 min|    9|
|       33 min|   16|
|      119 min|   38|
|21 Temporadas|    1|
|      479 min|    1|
|      165 min|    9|
|       12 min|    5|
|      142 min|   17|
|       30 min|   42|
|       27 min|   16|
|      157 min|   11|
|      173 min|    2|
|       39 min|   10|
|       82 min|  104|
|       21 min|   23|
|       40 min|   19|
+-------------+-----+
only showing top 20 rows



In [ ]:
amazon_df.show()

+---+-----+--------------------+--------------+---------------+--------------+-------------+-----------+--------------------+-----------+
| id| tipo|              titulo|          país|data_adicionado|ano_lancamento|classificacao|    duracao|              genero|      tempo|
+---+-----+--------------------+--------------+---------------+--------------+-------------+-----------+--------------------+-----------+
| s1|Filme|    A Grande Sedução|        Canada|     2021-03-30|          2014| Nao avaliado|    113 min|      Comedia, Drama|       113 |
| s2|Filme|  Cuide da boa noite|         India|     2021-03-30|          2018|          13+|    110 min|Drama, Internacional|       110 |
| s3|Filme|Segredos da Decepção|Estados Unidos|     2021-03-30|          2017| Nao avaliado|     74 min|Acao, Drama, Susp...|        74 |
| s4|Filme|Rosa: Mantendo-se...|Estados Unidos|     2021-03-30|          2014| Nao avaliado|     69 min|        Documentario|        69 |
| s5|Filme| Criador de Monstros|  

Criação da coluna que identifica a qual empresa pertence os dados

In [ ]:
valorA = 'Amazon'
valorN = 'Netflix'
amazon_df = amazon_df.withColumn("aplicativo", F.lit(valorA))

Convertendo o dataframe pyspark para pandas novamente para poder salvar a tabela com as novas alterações.

In [ ]:
amazon_conv =amazon_df.toPandas()

uma vez em pandas a tabela pode ser salva em csv

In [ ]:
amazon_conv.to_csv('Amazon_trat.csv',index=False)

uma outra forma de salvar o arquivo que foi usada para teste

In [ ]:
'''
(
    df1.write.format("csv")
    .option("header", "true")
    .option("inferschema", "true")
    .option("delimiter", ",")
    .save("/content/drive/MyDrive/teste_PF.csv")
)
'''

'\n(\n    df1.write.format("csv")\n    .option("header", "true")\n    .option("inferschema", "true")\n    .option("delimiter", ",")\n    .save("/content/drive/MyDrive/teste_PF.csv")\n)\n'

visualização da tabela nova

In [ ]:
teste=pd.read_csv('/content/Amazon_trat.csv')
teste

,id,tipo,titulo,país,data_adicionado,ano_lancamento,classificacao,duracao,genero,tempo,aplicativo
0,s1,Filme,A Grande Sedução,Canada,2021-03-30,2014,Nao avaliado,113 min,"Comedia, Drama",113,Amazon
1,s2,Filme,Cuide da boa noite,India,2021-03-30,2018,13+,110 min,"Drama, Internacional",110,Amazon
2,s3,Filme,Segredos da Decepção,Estados Unidos,2021-03-30,2017,Nao avaliado,74 min,"Acao, Drama, Suspense",74,Amazon
3,s4,Filme,Rosa: Mantendo-se Verdadeiro,Estados Unidos,2021-03-30,2014,Nao avaliado,69 min,Documentario,69,Amazon
4,s5,Filme,Criador de Monstros,Reino Unido,2021-03-30,1989,Nao avaliado,45 min,"Drama, Fantasia",45,Amazon
...,...,...,...,...,...,...,...,...,...,...,...
9663,s9664,Filme,Pride Of The Bowery,Nao Declarado,NaN,1940,7+,60 min,Comedia,60,Amazon
9664,s9665,Serie,Planet Patrol,Nao Declarado,NaN,2018,13+,4 Temporadas,Series,4 Temporadas,Amazon
9665,s9666,Filme,Outpost,Nao Declarado,NaN,2008,Restrito,90 min,Acao,90,Amazon
9666,s9667,Serie,Maradona: Blessed Dream,Nao Declarado,NaN,2021,Adultos,1 Temporada,"Drama, Esportes",1 Temporada,Amazon


# Criando uma terceira tabela usando concat

In [ ]:
import pandas as pd

Lendo as duas tabelas(Amazon e Netflix) no formato csv

In [ ]:
netflix_df = pd.read_csv('/content/ProjetoFinal_12/netflix_trat.csv')
amazon_df = pd.read_csv('/content/ProjetoFinal_12/Amazon_trat.csv')

Criando a tabela resultante da junção de dados da Amazon e da Netflix

In [ ]:
amazon_netflix = pd.concat([amazon_df, netflix_df])
amazon_netflix

,id,tipo,titulo,país,data_adicionado,ano_lancamento,classificacao,duracao,genero,tempo,aplicativo
0,s1,Filme,A Grande Sedução,Canada,2021-03-30,2014,Nao avaliado,113 min,"Comedia, Drama",113,Amazon
1,s2,Filme,Cuide da boa noite,India,2021-03-30,2018,13+,110 min,"Drama, Internacional",110,Amazon
2,s3,Filme,Segredos da Decepção,Estados Unidos,2021-03-30,2017,Nao avaliado,74 min,"Acao, Drama, Suspense",74,Amazon
3,s4,Filme,Rosa: Mantendo-se Verdadeiro,Estados Unidos,2021-03-30,2014,Nao avaliado,69 min,Documentario,69,Amazon
4,s5,Filme,Criador de Monstros,Reino Unido,2021-03-30,1989,Nao avaliado,45 min,"Drama, Fantasia",45,Amazon
...,...,...,...,...,...,...,...,...,...,...,...
8801,s8803,Filmes,Zodiac,Estados Unidos,2019-11-20,2007,Restrito,158 min,"Filmes de culto, dramas, suspense",NaN,Netflix
8802,s8804,Séries,Zombie Dumb,Nao Declarado,2019-07-01,2018,7+,2 Temporadas,"TV infantil, programas de TV coreanos, comédia...",NaN,Netflix
8803,s8805,Filmes,Zombieland,Estados Unidos,2019-11-01,2009,Restrito,88 min,"Comédias, Filmes de Terror",NaN,Netflix
8804,s8806,Filmes,Zoom,Estados Unidos,2020-01-11,2006,13+,88 min,"Filmes infantis e familiares, comédias",NaN,Netflix


Salvando a 3° tabela em csv


In [ ]:
amazon_netflix.to_csv('Amazon_Netflix.csv',index=False)

Transferindo dados para o Cloud storage

In [ ]:
!gsutil cp Amazon_netflix.csv gs://bucket-rosa/entrada/Amazon_netflix.csv

ServiceException: 401 Anonymous caller does not have storage.objects.list access to the Google Cloud Storage bucket.


************************************************************

# colab TerceiraTabela_Sql

In [ ]:
df = (spark.read.format("csv") \
  .option("inferSchema", "false") \
  .option("header", "true") \
  .option("sep", ",") \
  .load("/content/ProjetoFinal_12/Amazon_netflix.csv")
  .createOrReplaceTempView("app_video")
    )

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:

spark.sql("SELECT * FROM app_video").show()

+---+-----+--------------------+--------------+---------------+--------------+-------------+-----------+--------------------+----------+
| id| tipo|              titulo|          país|data_adicionado|ano_lancamento|classificacao|    duracao|              genero|aplicativo|
+---+-----+--------------------+--------------+---------------+--------------+-------------+-----------+--------------------+----------+
| s1|Filme|    A Grande Sedução|        Canada|     2021-03-30|          2014| Nao avaliado|    113 min|      Comedia, Drama|    Amazon|
| s2|Filme|  Cuide da boa noite|         India|     2021-03-30|          2018|          13+|    110 min|Drama, Internacional|    Amazon|
| s3|Filme|Segredos da Decepção|Estados Unidos|     2021-03-30|          2017| Nao avaliado|     74 min|Acao, Drama, Susp...|    Amazon|
| s4|Filme|Rosa: Mantendo-se...|Estados Unidos|     2021-03-30|          2014| Nao avaliado|     69 min|        Documentario|    Amazon|
| s5|Filme| Criador de Monstros|   Reino 

As consultas em sparkSql foram implementadas em um novo colab (TerceiraTabela_Sql.ipynb)
